In [3]:
import numpy as np
import pandas as pd

# Experiment 1 -  Gravity Drop-Test 

In the theoretical solution interest lays only in the Z position and Z linear velocity.

In [4]:
def run_gravity_drop_test(intial_z_position, total_experiment_time, destination_file_name):
    # Initial state
    observation_times = [0]
    z_velocities = [0]
    z_positions = [intial_z_position]

    g = -9.81
    crt_time = 0.0

    # Simulate the cube falling until it touches the ground. 
    # Note that since the dimensions of the cube are 1x1x1, in order to touch the ground, the cube's center of mass Z position should be 0.5
    while crt_time < total_experiment_time:
        if intial_z_position + (g*(crt_time**2))/2 <= 0.5:
            break
        crt_time += 0.001
        z_velocities.append(g * crt_time)
        z_positions.append(intial_z_position + (g*(crt_time**2))/2)
        observation_times.append(crt_time)

    # Record the stopping time and maximum linear velocity of the cube.
    stopping_time = crt_time
    maximum_velocity = z_velocities[-1]

    crt_time += 0.001
    # Complete the observations array to be consistent with the simulation duration in Gazebo and Unreal.
    while crt_time < total_experiment_time:
        z_velocities.append(0)
        z_positions.append(0.5)
        observation_times.append(crt_time)
        crt_time += 0.001

    # Load the data into a pandas data frame to add column labels.
    data_frame = pd.DataFrame.from_dict({'Time': np.transpose(observation_times), 
                                'Z Velocity': np.transpose(z_velocities), 
                                'Z Position': np.transpose(z_positions)})

    data_frame.to_csv(destination_file_name, index=False)

In [9]:
run_gravity_drop_test(10, 2, '10M_Gravity_Drop.csv')
run_gravity_drop_test(50, 4, '50M_Gravity_Drop.csv')
run_gravity_drop_test(75, 4, '75M_Gravity_Drop.csv')
run_gravity_drop_test(100, 5, '100M_Gravity_Drop.csv')

# Experiment 2 - Impulse Applied to a Cube

Force is applied on the positive Y-axis, therefore only interesest is in Y position and Y linear velocity. Note that this experiment contains two sub-scenarios, neglecting and including dynamic friction.

## Experiment 2.1 - Neglecting dynamic friction


In [12]:
applied_forces = [1000, 10000]
impulse_duration = 0.001 # This represents the duration for which the impulse is applied.
total_experiment_time = 4.0
cube_mass = 1


# Get the theoretical solutions for all the forces considered.
for crt_force in applied_forces:
    # Initial state
    observation_times = []
    y_velocities = []
    y_positions = []
    crt_time = 0
    
    # Avoid overstepping the total time by 0.001
    while crt_time < total_experiment_time - 0.001:
        crt_time += 0.001
    
        crt_y_velocity = (crt_force * impulse_duration)/cube_mass
        y_velocities.append(crt_y_velocity)
        y_positions.append(crt_y_velocity * crt_time)
        observation_times.append(crt_time)
        
    data_frame = pd.DataFrame.from_dict({'Time': np.transpose(observation_times), 
                            'Y Velocity': np.transpose(y_velocities), 
                            'Y Position': np.transpose(y_positions)})

    data_frame.to_csv(str(crt_force) + "N Impulse_No_Friction.csv", index=False) 


## Experiment 2.2 - Including dynamic friction


In [5]:
def compute_position_integration(initial_velocity, g, friction_coefficient, t):
    # This function computes the integral from 0 to t for the position of the cube.
    # From theoretical results presented in section 3.3.2., the position is given by:
    # Position = integral(0->t)(v0 - g * μ * t)dt
    return initial_velocity * t - (g * friction_coefficient * (t**2))/2

applied_forces = [1000, 10000]
impulse_duration = 0.001 # This represents the duration for which the impulse is applied.
total_experiment_time = 4.0
cube_mass = 1
g = 9.81
friction_coefficient = 1

# Get the theoretical solutions for all the forces considered.
for crt_force in applied_forces:
    # Initial state
    observation_times = []
    y_velocities = []
    y_positions = []
    crt_time = 0
    
    # Stage 1: Compute the velocity and position immediately after applying the force.
    while crt_time < impulse_duration:
        crt_time += 0.001
        crt_y_velocity = (crt_force * impulse_duration)/cube_mass
        y_velocities.append(crt_y_velocity)
        y_positions.append(crt_y_velocity * crt_time)
        observation_times.append(crt_time)
        
    initial_velocity = y_velocities[-1]
    
    # Stage 2: Compute the velocity and position as friction starts acting
    # Avoid overstepping the total time by 0.001
    while crt_time < total_experiment_time - 0.001:
        crt_time += 0.001
    
        crt_y_velocity = initial_velocity - (g * friction_coefficient * crt_time)
        if crt_y_velocity > 0:
            y_velocities.append(crt_y_velocity)
            y_positions.append(compute_position_integration(initial_velocity, g, friction_coefficient, crt_time))
        else:
            y_velocities.append(0)
            y_positions.append(y_positions[-1])

        observation_times.append(crt_time)
        
    data_frame = pd.DataFrame.from_dict({'Time': np.transpose(observation_times), 
                            'Y Velocity': np.transpose(y_velocities), 
                            'Y Position': np.transpose(y_positions)})

    data_frame.to_csv(str(crt_force) + "N Impulse_With_Friction.csv", index=False) 


# Experiment 3 - Constant Force Applied to a Cube

In [24]:
def compute_force_position_integration(F, g, friction_coefficient, m, t):
    # This function computes the integral from 0 to t for the position of the cube.
    # From theoretical results presented in section 3.4., the position is given by:
    # Position = integral(0->t)(1/m * (F- m * g * μ) * t)dt
    return ((F - m * g * friction_coefficient) * (t **2))/(2 * m)

# Note that this implementation is extremely rudimentary and only works correctly for forces that are larger than m * g * μ
applied_forces = [10, 20]
total_experiment_time = 10.0
force_applying_time = 5.0

m = 1
g = 9.81
friction_coefficient = 1

for crt_force in applied_forces:
    observation_times = []
    y_velocities = []
    y_positions = []
    crt_time = 0
    
    # Phase 1 - apply the force constantly for 5 seconds.
    while crt_time < force_applying_time:
        crt_time += 0.001
        crt_y_velocity = 1/m * (crt_force - m * g * friction_coefficient)*crt_time
        y_velocities.append(crt_y_velocity)
        y_positions.append(compute_force_position_integration(crt_force, g, friction_coefficient, m, crt_time))
        observation_times.append(crt_time)
    
    # Phase 2 - no force being applied, velocity starts decreasing due to friction.
    initial_velocity = y_velocities[-1]
    position = y_positions[-1]
    while crt_time < total_experiment_time - 0.001:
        crt_time += 0.001
        crt_y_velocity = initial_velocity - (g * friction_coefficient * (crt_time - force_applying_time))
        if crt_y_velocity > 0:
            y_velocities.append(crt_y_velocity)
            y_positions.append(position + compute_position_integration(initial_velocity, g, friction_coefficient, (crt_time - force_applying_time)))
        else:
            y_velocities.append(0)
            y_positions.append(y_positions[-1])

        observation_times.append(crt_time)
        
    data_frame = pd.DataFrame.from_dict({'Time': np.transpose(observation_times), 
                            'Y Velocity': np.transpose(y_velocities), 
                            'Y Position': np.transpose(y_positions)})

    data_frame.to_csv(str(crt_force) + "N Constant Force.csv", index=False) 
